# Determining the origin of the broad component in "normal" star-forming galaxies using a matched-sample analysis
---
From Kaasisnen et al. (2017):
> *We select three ‘matched’ samples from the full local sample by matching **an ensemble of local counterparts to each galaxy in our z ∼ 1.5 [O II]-Hα sample**. Our first local comparison sample is matched solely on stellar mass, without applying any constraints to the SFR. We refer to this sample as the $M_∗$-matched sample...  To create the $M_∗$-matched sample, **we require the stellar mass of the local counterparts to be within 0.2 dex of each high-z galaxy.** Conversely, for our second local comparison sample we require the SFRs of the high-z galaxies and local counterparts to be consistent within 0.2 dex but impose no constraints on $M_∗$...*

> *To ensure that the statistical properties of the matched, local and high-z samples are equivalent, **we select the same number of local counterparts for each high-z galaxy**. Although there were more than 50 local galaxies with equivalent $M_∗$ for each high-z galaxy, a greater sample size did not result in a change in the electron-density distribution. **We therefore limit the size of our $M_∗$-matched sample, by randomly selecting 50 local galaxies for each high-z galaxy**. In contrast, the number of local counterparts in both the SFR-matched and the $M_∗$-and-SFR-matched samples is limited by the rarity of high SFR galaxies in the local SDSS sample. We only find seven local counterparts for our highest SFR high-z galaxy and thus select seven local galaxies at random for the remainder of our sample. Because we impose further constraints to select the $M_∗$-and-SFR- matched sample we are limited to five local counterparts for each high-z galaxy.*

To summarise, we should
* Determine how to quantify our independent variable (i.e. how to classify galaxies into having >1 component)
* Determine which properties on which to match our sample - e.g. stellar mass, SFR, morphology, etc. 
* After this, look at the sample sizes: how many 1-comp galaxies should we match to each 2-comp galaxy?

In [1]:
%matplotlib widget

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from astropy.visualization import hist

from spaxelsleuth.loaddata.sami import load_sami_df
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, ncomponents_colours

import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [3]:
savefigs = True
fig_path = os.path.join(os.environ["SAMI_FIG_DIR"], "presentations")
# fig_path = os.path.join(os.environ["SAMI_DIR"], "matched_samples")

In [4]:
ncomponents = "recom"
bin_type = "default"
eline_SNR_min = 5

In [5]:
# Load the metadata
df_metadata = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], "sami_dr3_metadata.hd5"))
df_metadata_extended = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], "sami_dr3_metadata_extended.hd5"))

# Mask out the cell with the extremely high R_e (which is almost certainly incorrect!)
cond_bad_Re = df_metadata_extended["R_e (kpc)"] > 500
df_metadata_extended.loc[cond_bad_Re, "R_e (kpc)"] = np.nan
assert not any(df_metadata_extended.index.duplicated())

# Add data from the EmissionLine1compDR3 table
df_ap1comp = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], f"sami_apertures_extcorr_minSNR=5.hd5"))
# data_path = os.path.join(spaxelsleuth.loaddata.__file__.split("loaddata")[0], "data")
# df_ap1comp = pd.read_csv(os.path.join(data_path, "sami_EmissionLine1compDR3.csv"))
# df_ap1comp = df_ap1comp.set_index("catid").drop(["Unnamed: 0"], axis=1)
# df_ap1comp = df_ap1comp[~df_ap1comp.index.duplicated(keep='first')]  # Remove duplicate indices
assert not any(df_ap1comp.index.duplicated())

# Merge onto df_metadata_extended
df_metadata_extended = df_metadata_extended.merge(df_ap1comp[[c for c in df_ap1comp.columns if c not in df_metadata_extended.columns]], how="inner", left_index=True, right_index=True)

# Compute the mean SFR surface density within a 3kpc aperture 
df_metadata_extended["sigma_SFR (3kpc round)"] = df_metadata_extended["SFR (3kpc round)"] / (np.pi * 3**2)
df_metadata_extended["sigma_SFR (3kpc round) (error)"] = df_metadata_extended["SFR error (3kpc round)"] / (np.pi * 3**2)
df_metadata_extended["log sigma_SFR (3kpc round)"] = np.log10(df_metadata_extended["sigma_SFR (3kpc round)"])  # not bothering with computing log errors

# Specific SFR in 3kpc
df_metadata_extended["log sSFR (3kpc round)"] = np.log10(df_metadata_extended["SFR (3kpc round)"]) - df_metadata_extended["mstar"]

# Compute the mean SFR surface density within a 3kpc aperture 
df_metadata_extended["sigma_SFR (R_e)"] = df_metadata_extended["SFR (R_e)"] / (np.pi * df_metadata_extended["R_e (kpc)"]**2)
df_metadata_extended["sigma_SFR (R_e) (error)"] = df_metadata_extended["SFR error (R_e)"] / (np.pi * df_metadata_extended["R_e (kpc)"]**2)
df_metadata_extended["log sigma_SFR (R_e)"] = np.log10(df_metadata_extended["sigma_SFR (R_e)"])  # not bothering with computing log errors

# Specific SFR in 3kpc
df_metadata_extended["log sSFR (R_e)"] = np.log10(df_metadata_extended["SFR (R_e)"]) - df_metadata_extended["mstar"]


assert not any(df_ap1comp.index.duplicated())


In [6]:
df = load_sami_df(ncomponents=ncomponents, bin_type=bin_type, eline_SNR_min=eline_SNR_min, correct_extinction=True)

In load_sami_df(): Loading DataFrame...
In load_sami_df(): Finished!


### STEP 1: quantifying the variable of interest
--- 
Classify "2-component galaxies" as being galaxies in which >30% of spaxels have 2 components, and >90% of spaxels are star-forming. Maybe also remove Seyfert galaxies. 

Note that we should only use the spaxels in which the final number of components matches the original number of components.

Then, we should do a couple of checks before proceeding:
* how many galaxies are there that meet these criteria?
* plot the sSFR of these galaxies vs. stellar mass in comparison with the whole sample 

In [7]:
# Limit sample to having high-quality multi-component fits
df_hq = df[df["Number of components"] == df["Number of components (original)"]]
print(f"{df_hq.shape[0] / df.shape[0] * 100}% of spaxels have high-quality components only")

61.04448405265259% of spaxels have high-quality components only


In [8]:
# Select 1/2-component galaxies
gals = df_hq.catid.unique()
gals_2comp = []
gals_1comp = []
for gal in tqdm(gals):
    df_gal = df_hq[df_hq.catid == gal]
    
    # Criterion: >20% of spaxels must have >1 component 
    cond_anycomp = df_gal["Number of components"] >= 1
    n_anycomp = df_gal[cond_anycomp].shape[0]
    cond_2comp = df_gal["Number of components"] >= 2
    n_2comp = df_gal[cond_2comp].shape[0]
    
    # Criterion: of those spaxels that can be classified, >80% must be SF 
    cond_SF = df_gal["BPT (total)"] == "SF"
    cond_Seyfert = df_gal["BPT (total)"] == "Seyfert"
    cond_LINER = df_gal["BPT (total)"] == "LINER"
    cond_any = df_gal["BPT (total)"] != "Not classified"
    n_SF = df_gal[cond_SF].shape[0]
    n_Seyfert = df_gal[cond_Seyfert].shape[0]
    n_LINER = df_gal[cond_LINER].shape[0]
    n_anycat = df_gal[cond_any].shape[0]
    
    if n_anycomp > 0 and n_anycat > 50 and n_Seyfert == 0 and n_LINER == 0:
        if (n_SF / n_anycat) > 0.8:
            if ((n_2comp / n_anycomp) > 0.2):
                gals_2comp.append(gal)
            elif ((n_2comp / n_anycomp) < 0.01):
                gals_1comp.append(gal)

print(f"{len(gals_2comp)}/{len(gals)} meet our 2-component criteria")
print(f"{len(gals_1comp)}/{len(gals)} meet our 1-component criteria")

100%|██████████| 2997/2997 [00:33<00:00, 90.73it/s]

102/2997 meet our 2-component criteria
442/2997 meet our 1-component criteria


## Step 0a: is there a correlation between $\Sigma_{\rm SFR}$ and the *number* of 2-component spaxels?

In [14]:
# Select 1/2-component galaxies
gals = df_hq.catid.unique()
gals_SF = []
for gal in tqdm(gals):
    df_gal = df_hq[df_hq.catid == gal]
    
    # Count the number of spaxels in each galaxy with 1, 2 or 3 components
    cond_anycomp = df_gal["Number of components"] >= 1
    n_anycomp = df_gal[cond_anycomp].shape[0]
    cond_1comp = df_gal["Number of components"] == 1
    n_1comp = df_gal[cond_1comp].shape[0]
    cond_2comp = df_gal["Number of components"] == 2
    n_2comp = df_gal[cond_2comp].shape[0]
    cond_3comp = df_gal["Number of components"] == 3
    n_3comp = df_gal[cond_3comp].shape[0]
    cond_23comp = cond_2comp | cond_3comp
    n_23comp = df_gal[cond_23comp].shape[0]
    df_metadata_extended.loc[gal, "Number of any-component spaxels"] = n_anycomp
    df_metadata_extended.loc[gal, "Number of 1-component spaxels"] = n_1comp
    df_metadata_extended.loc[gal, "Number of 2-component spaxels"] = n_2comp
    df_metadata_extended.loc[gal, "Number of 3-component spaxels"] = n_3comp
    df_metadata_extended.loc[gal, "Number of >1-component spaxels"] = n_23comp
    
    # Compute the mean SFR surface density in spaxels with 1, 2 or 3 components 
    df_metadata_extended.loc[gal, "Mean log SFR surface density in 1-component spaxels"] = df_gal.loc[cond_1comp, "log SFR surface density (total)"].mean()
    df_metadata_extended.loc[gal, "Mean log SFR surface density in 2-component spaxels"] = df_gal.loc[cond_2comp, "log SFR surface density (total)"].mean()
    df_metadata_extended.loc[gal, "Mean log SFR surface density in 3-component spaxels"] = df_gal.loc[cond_3comp, "log SFR surface density (total)"].mean()
    df_metadata_extended.loc[gal, "Mean log SFR surface density in >1-component spaxels"] = df_gal.loc[cond_23comp, "log SFR surface density (total)"].mean()
    
    # Criterion: of those spaxels that can be classified, >80% must be SF 
    cond_SF = df_gal["BPT (total)"] == "SF"
    cond_Seyfert = df_gal["BPT (total)"] == "Seyfert"
    cond_LINER = df_gal["BPT (total)"] == "LINER"
    cond_any = df_gal["BPT (total)"] != "Not classified"
    n_SF = df_gal[cond_SF].shape[0]
    n_Seyfert = df_gal[cond_Seyfert].shape[0]
    n_LINER = df_gal[cond_LINER].shape[0]
    n_anycat = df_gal[cond_any].shape[0]
    
    if n_anycat > 50 and n_Seyfert == 0 and n_LINER == 0:
        if (n_SF / n_anycat) > 0.8:
            gals_SF.append(gal)

print(f"{len(gals_SF)}/{len(gals)} are classified as star-forming")

100%|██████████| 2997/2997 [00:58<00:00, 50.85it/s]

855/2997 are classified as star-forming


In [16]:
# Scatter plot: SFR surface density in 2-component spaxels as a function of % of 2-component spaxels 
nn = ">1"
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.scatter(x=df_metadata_extended.loc[gals, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals, "Number of any-component spaxels"], 
           y=df_metadata_extended.loc[gals, f"Mean log SFR surface density in {nn}-component spaxels"], 
           color="lightgrey", label="Full sample", s=2)
m = ax.scatter(x=df_metadata_extended.loc[gals_SF, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals_SF, "Number of any-component spaxels"], 
           y=df_metadata_extended.loc[gals_SF, f"Mean log SFR surface density in {nn}-component spaxels"], 
           color="blue",
           label="Star-forming galaxies", s=10)
ax.set_ylabel(r"Average $\Sigma_{\rm SFR}$ in " + f"{nn}" + r"-component spaxels ($\log [\rm M_\odot \, yr^{-1} \, kpc^{-2}]$)")
ax.set_xlabel("Fraction of spaxels with >1 components")
ax.set_xscale("log")
ax.set_ylim([-3.75, -0.0])
ax.set_xlim([1e-3, 2])
ax.legend(loc="lower right", fontsize=10)
ax.grid()

# Spearman's rank correlation coefficient
from scipy.stats import ks_2samp, anderson_ksamp, spearmanr
x = df_metadata_extended.loc[gals, "Number of >1-component spaxels"].values / df_metadata_extended.loc[gals, "Number of any-component spaxels"].values
y = df_metadata_extended.loc[gals, f"Mean log SFR surface density in {nn}-component spaxels"]
good_pts = ~np.isnan(x) & ~np.isnan(y)
res = spearmanr(x[good_pts], y[good_pts])

corr = res.correlation
pval = res.pvalue
ax.text(s=f"Spearman correlation coefficient: {corr:.3f}" + "\n" + f"($p = ${pval:.4g})", x=0.02, y=0.98, verticalalignment="top", transform=ax.transAxes) 

if savefigs:
    fname = os.path.join(fig_path, f"avg_sfrdense_2ndcomp_vs_2comp_frac_sf_gals.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/avg_sfrdense_2ndcomp_vs_2comp_frac_sf_gals.pdf


In [20]:
# Scatter plot: SFR surface density in 2-component spaxels as a function of % of 2-component spaxels 
nn = ">1"
fig, ax = plt.subplots(nrows=1, ncols=1)
bbox = ax.get_position()
cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.05, bbox.height])
ax.scatter(x=df_metadata_extended.loc[gals, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals, "Number of any-component spaxels"], 
           y=df_metadata_extended.loc[gals, f"Mean log SFR surface density in {nn}-component spaxels"], 
           color="lightgrey", label="Full sample", s=2)
m = ax.scatter(x=df_metadata_extended.loc[gals_SF, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals_SF, "Number of any-component spaxels"], 
           y=df_metadata_extended.loc[gals_SF, f"Mean log SFR surface density in {nn}-component spaxels"], 
           # color="blue",
           c=df_metadata_extended.loc[gals_SF, f"Mean log SFR surface density in {nn}-component spaxels"] - df_metadata_extended.loc[gals_SF, "log sigma_SFR (R_e)"],
           cmap="coolwarm_r", vmin=-0.5, vmax=0.5,
           label="Star-forming galaxies", s=10)
plt.colorbar(mappable=m, cax=cax)
cax.set_ylabel(r"$\langle \Sigma_{\rm SFR,\,>1-comp} \rangle - \langle \Sigma_{{\rm SFR},\,1R_e} \rangle$")
ax.set_ylabel(r"Average $\Sigma_{\rm SFR}$ in " + f"{nn}" + r"-component spaxels ($\log [\rm M_\odot \, yr^{-1} \, kpc^{-2}]$)")
ax.set_xlabel("Fraction of spaxels with >1 components")
ax.set_xscale("log")
ax.set_ylim([-3.75, -0.0])
ax.set_xlim([1e-3, 2])
ax.legend(loc="lower right", fontsize=10)
ax.grid()

# Spearman's rank correlation coefficient
from scipy.stats import ks_2samp, anderson_ksamp, spearmanr
x = df_metadata_extended.loc[gals, "Number of >1-component spaxels"].values / df_metadata_extended.loc[gals, "Number of any-component spaxels"].values
y = df_metadata_extended.loc[gals, f"Mean log SFR surface density in {nn}-component spaxels"]
good_pts = ~np.isnan(x) & ~np.isnan(y)
res = spearmanr(x[good_pts], y[good_pts])

corr = res.correlation
pval = res.pvalue
ax.text(s=f"Spearman correlation coefficient: {corr:.3f}" + "\n" + f"($p = ${pval:.4g})", x=0.02, y=0.98, verticalalignment="top", transform=ax.transAxes) 

if savefigs:
    fname = os.path.join(fig_path, f"avg_sfrdense_2ndcomp_vs_2comp_frac_sf_gals_coloured.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/avg_sfrdense_2ndcomp_vs_2comp_frac_sf_gals_coloured.pdf


In [22]:
# Scatter plot: fraction of 2-component spaxels vs. SFR surface density 
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.scatter(y=df_metadata_extended.loc[gals, "log sigma_SFR (R_e)"], 
           x=df_metadata_extended.loc[gals, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals, "Number of any-component spaxels"], 
           color="lightgrey", label="Full sample", s=2)
ax.scatter(y=df_metadata_extended.loc[gals_SF, "log sigma_SFR (R_e)"], 
           x=df_metadata_extended.loc[gals_SF, "Number of >1-component spaxels"] / df_metadata_extended.loc[gals_SF, "Number of any-component spaxels"], 
           color="blue", label="Star-forming galaxies", s=10)
ax.set_ylabel(label_fn("SFR surface density"))
ax.set_xlabel("Fraction of spaxels with >1 components")
ax.set_xscale("log")
ax.set_ylim([-3.75, -0.0])
ax.set_xlim([1e-3, 2])
ax.legend(loc="lower right", fontsize=10)
ax.grid()

# Spearman's rank correlation coefficient
from scipy.stats import ks_2samp, anderson_ksamp, spearmanr
y = df_metadata_extended.loc[gals_SF, "log sigma_SFR (R_e)"].values
x = df_metadata_extended.loc[gals_SF, "Number of >1-component spaxels"].values / df_metadata_extended.loc[gals_SF, "Number of any-component spaxels"].values
good_pts = ~np.isnan(x) & ~np.isnan(y)
res = spearmanr(x[good_pts], y[good_pts])

corr = res.correlation
pval = res.pvalue
ax.text(s=f"Spearman correlation coefficient: {corr:.3f}" + "\n" + f"($p = ${pval:.4g})", x=0.02, y=0.98, verticalalignment="top", transform=ax.transAxes) 

if savefigs:
    fname = os.path.join(fig_path, f"sfrdense_1re_vs_2comp_frac_sf_gals.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/sfrdense_1re_vs_2comp_frac_sf_gals.pdf


### Step 0b: what is the fraction of 2-component spaxels in bins of $\Sigma_{\rm SFR}$?

In [ ]:
sigma_sfr_vals = np.linspace(-5.5, 0.0, 12)
sigma_sfr_midpts = [0.5 * (sigma_sfr_vals[ii] + sigma_sfr_vals[ii + 1]) for ii in range(len(sigma_sfr_vals) - 1)]
mean_3comp_frac_vals = np.zeros(len(sigma_sfr_vals) - 1)
mean_2comp_frac_vals = np.zeros(len(sigma_sfr_vals) - 1)
mean_1comp_frac_vals = np.zeros(len(sigma_sfr_vals) - 1)
for ii in range(len(sigma_sfr_vals) - 1):
    cond = df_hq["BPT (total)"] == "SF"
    cond &= (df_hq["log SFR surface density (total)"] >= sigma_sfr_vals[ii]) & (df_hq["log SFR surface density (total)"] < sigma_sfr_vals[ii + 1])
    if any(cond):
        cond_3comp = cond & (df_hq["Number of components"] == 3)
        cond_2comp = cond & (df_hq["Number of components"] == 2)
        cond_1comp = cond & (df_hq["Number of components"] == 1)
        mean_3comp_frac_vals[ii] = df_hq.loc[cond_3comp].shape[0] / df_hq.loc[cond].shape[0]
        mean_2comp_frac_vals[ii] = df_hq.loc[cond_2comp].shape[0] / df_hq.loc[cond].shape[0]
        mean_1comp_frac_vals[ii] = df_hq.loc[cond_1comp].shape[0] / df_hq.loc[cond].shape[0]
    else:
        mean_3comp_frac_vals[ii] = np.nan
        mean_2comp_frac_vals[ii] = np.nan
        mean_1comp_frac_vals[ii] = np.nan


In [ ]:
fig, ax = plt.subplots()
ax.plot(sigma_sfr_midpts, mean_1comp_frac_vals, "d-", label="1 component", lw=0.5, color=ncomponents_colours[1])
ax.plot(sigma_sfr_midpts, mean_2comp_frac_vals, "d-", label="2 components", color=ncomponents_colours[2])
ax.plot(sigma_sfr_midpts, mean_3comp_frac_vals, "d-", label="3 components", lw=0.5, color=ncomponents_colours[3])
ax.set_xlabel(label_fn("SFR surface density"))
ax.set_ylabel("Fraction of spaxels with 2 components")
ax.set_yscale("log")
ax.set_xlim([-3.75, -0.0])
ax.set_ylim([1e-4, 2])
ax.legend(loc="lower right", fontsize=10)
ax.grid()

if savefigs:
    fname = os.path.join(fig_path, f"2comp_spaxel_fraction_sf_spaxels.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

### Step 0c: *within individual galaxies*, do 2-component spaxels have higher SFR surface densities? & are these blue/redshifted?

In [ ]:
plt.close("all")

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages(os.path.join(fig_path, f"2comp_spaxel_fraction_sf_galaxies.pdf"))

for gal in gals_SF:
    df_gal = df_hq[df_hq.catid == gal]

    # Limit to star-forming spaxels 
    cond_1comp = (df_gal["BPT (total)"] == "SF") & (df_gal["Number of components"] == 1)
    cond_2comp = (df_gal["BPT (total)"] == "SF") & (df_gal["Number of components"] == 2)
    cond_3comp = (df_gal["BPT (total)"] == "SF") & (df_gal["Number of components"] == 3)
    
    # Plot 
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    plot_sdss_image(df_gal=df_gal, ax=axs[0])
    plot2dmap(df_gal=df_gal, bin_type="default", survey="sami", col_z="Number of components", ax=axs[1], plot_colorbar=True, cax_orientation="horizontal")
    hist(df_gal.loc[cond_1comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="stepfilled", alpha=0.3, color=ncomponents_colours[1])
    hist(df_gal.loc[cond_1comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="step", color=ncomponents_colours[1], label="1 component")
    hist(df_gal.loc[cond_2comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="stepfilled", alpha=0.3, color=ncomponents_colours[2])
    hist(df_gal.loc[cond_2comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="step", color=ncomponents_colours[2], label="2 components")
    hist(df_gal.loc[cond_3comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="stepfilled", alpha=0.3, color=ncomponents_colours[3])
    hist(df_gal.loc[cond_3comp, "log SFR surface density (total)"], range=(-4.0, 0.0), ax=axs[2], bins=20, histtype="step", color=ncomponents_colours[3], label="3 components")
    axs[2].set_xlabel(label_fn("log SFR surface density"))
    axs[2].set_ylabel(r"$N$")
    axs[2].legend(loc="lower center", fontsize=10, bbox_to_anchor=[0.5, 1.01])
    axs[2].grid(alpha=0.4)
    
    if savefigs:
       pp.savefig(fig, bbox_inches="tight")
        
    plt.close("all")
pp.close()


### Step 0d: are the SFR surface densitites in 2-component galaxies *always* high, or is this only the case in galaxies with lots of 2-component spaxels? 

In [ ]:
for gal in gals_SF:
    df_gal = df_hq[df_hq["catid"] == gal]
    
    # Compute the mean SFR surface density in the 2-component spaxels 
    
    
    

### Step 1a: what are the general properties of the 1- and 2-comp. galaxies?
---

In [10]:
fig, ax = plt.subplots()
hist(df_metadata_extended["log sigma_SFR (R_e)"], range=(-5, 0), bins=30, ax=ax, histtype="step", color="grey", label="Full sample")
hist(df_metadata_extended.loc[gals_1comp, "log sigma_SFR (R_e)"], range=(-5, 0), histtype="stepfilled", color="orange", alpha=0.3, bins=30, ax=ax, label="1-component galaxies")
hist(df_metadata_extended.loc[gals_2comp, "log sigma_SFR (R_e)"], range=(-5, 0), histtype="step", color="red", bins=30,  ax=ax, label="2-component galaxies")
ax.legend()
ax.set_ylabel(r"$N$")
ax.set_xlabel(r"$\log \left(\Sigma_{\rm SFR,\,1R_e}\,[\rm M_\odot\,yr^{-1}\,kpc^{-2}]\right)$")
ax.grid()

if savefigs:
    fname = os.path.join(fig_path, f"matched_sample_hist.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_hist.pdf


In [38]:
fig, axs = plt.subplots(nrows=3, ncols=2, squeeze=False, sharex="col", sharey="row", figsize=(12, 12))
fig.subplots_adjust(hspace=0, wspace=0)
axs[0][1].set_visible(False)
ap = "R_e"
df_metadata_extended[f"log SFR ({ap})"] = np.log10(df_metadata_extended[f"SFR ({ap})"])
df_metadata_extended["log SFR (total)"] = np.log10(df_metadata_extended["SFR (total)"])

# Resize axes 
bbox = axs[0][0].get_position()
axs[0][0].set_position([bbox.x0, bbox.y0, bbox.width, 0.5 * bbox.height])
bbox = axs[1][1].get_position()
axs[1][1].set_position([bbox.x0, bbox.y0, bbox.width * 0.3, bbox.height])
bbox = axs[2][1].get_position()
axs[2][1].set_position([bbox.x0, bbox.y0, bbox.width * 0.3, bbox.height])


col_x = "mstar"
hist(df_metadata_extended[col_x], range=(7, 12), bins=30, ax=axs[0][0], histtype="step", color="grey", label="Full sample")
hist(df_metadata_extended.loc[gals_1comp, col_x], range=(7, 12), histtype="stepfilled", color="orange", alpha=0.3, bins=30, ax=axs[0][0], label="1-component galaxies")
hist(df_metadata_extended.loc[gals_2comp, col_x], range=(7, 12), histtype="step", color="red", bins=30,  ax=axs[0][0], label="2-component galaxies")
axs[0][0].set_ylabel(r"$N$")

col_y = f"log SFR ({ap})"
axs[1][0].scatter(df_metadata_extended[col_x], df_metadata_extended[col_y], c="lightgrey", s=5, label="Full sample")
axs[1][0].scatter(df_metadata_extended.loc[gals_1comp, col_x], df_metadata_extended.loc[gals_1comp, col_y], c="orange", alpha=0.4, s=20, label="1-component galaxies")
axs[1][0].scatter(df_metadata_extended.loc[gals_2comp, col_x], df_metadata_extended.loc[gals_2comp, col_y], c="w", edgecolors="r", s=20, label="2-component galaxies")
axs[1][0].set_xlabel(label_fn(col_x))
axs[1][0].set_ylabel(r"$\log {\rm SFR}\,(1\,R_e)$")
hist(df_metadata_extended[col_y], range=axs[1][1].get_ylim(), bins=30, ax=axs[1][1], histtype="step", color="grey", label="Full sample", orientation="horizontal")
hist(df_metadata_extended.loc[gals_1comp, col_y], range=axs[1][1].get_ylim(), histtype="stepfilled", color="orange", alpha=0.3, bins=30, ax=axs[1][1], label="1-component galaxies", orientation="horizontal")
hist(df_metadata_extended.loc[gals_2comp, col_y], range=axs[1][1].get_ylim(), histtype="step", color="red", bins=30,  ax=axs[1][1], label="2-component galaxies", orientation="horizontal")


col_y = f"log sigma_SFR ({ap})"
axs[2][0].scatter(df_metadata_extended[col_x], df_metadata_extended[col_y], c="lightgrey", s=5, label="Full sample")
axs[2][0].scatter(df_metadata_extended.loc[gals_1comp, col_x], df_metadata_extended.loc[gals_1comp, col_y], c="orange", alpha=0.4, s=20, label="1-component galaxies")
axs[2][0].scatter(df_metadata_extended.loc[gals_2comp, col_x], df_metadata_extended.loc[gals_2comp, col_y], c="w", edgecolors="r", s=20, label="2-component galaxies")
axs[2][0].set_xlabel(label_fn(col_x))
axs[2][0].set_ylabel(r"$\log \Sigma_{\rm SFR}\,(1\,R_e)$")
hist(df_metadata_extended[col_y], range=axs[2][0].get_ylim(), bins=30, ax=axs[2][1], histtype="step", color="grey", label="Full sample", orientation="horizontal")
hist(df_metadata_extended.loc[gals_1comp, col_y], range=axs[2][0].get_ylim(), histtype="stepfilled", color="orange", alpha=0.3, bins=30, ax=axs[2][1], label="1-component galaxies", orientation="horizontal")
hist(df_metadata_extended.loc[gals_2comp, col_y], range=axs[2][0].get_ylim(), histtype="step", color="red", bins=30,  ax=axs[2][1], label="2-component galaxies", orientation="horizontal")
axs[2][1].set_xlabel(r"$N$")

axs[1][0].legend(loc="lower right")
for ax in axs.flat:
    ax.grid()

# col_y = f"log sSFR ({ap})"
# axs[0][2].scatter(df_metadata_extended.loc[gals_1comp, col_x], df_metadata_extended.loc[gals_1comp, col_y], c="orange", alpha=0.4, s=20)
# axs[0][2].scatter(df_metadata_extended.loc[gals_2comp, col_x], df_metadata_extended.loc[gals_2comp, col_y], c="w", edgecolors="r", s=20)
# axs[0][2].set_xlabel(label_fn(col_x))
# axs[0][2].set_ylabel(label_fn(col_y))

if savefigs:
    fname = os.path.join(fig_path, f"matched_sample_scatter_plots.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_scatter_plots.pdf


In [42]:
plt.close("all")

In [41]:
# Look at the 2-component galaxies 
N = 1
for gal in gals_2comp[:N]:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))
    bbox = axs[-2].get_position()
    axs[-1].set_position([bbox.x0 + bbox.width, bbox.y0, bbox.width, bbox.height])
    df_gal = df_hq[df_hq.catid == gal]
    ax_sdss = plot_sdss_image(df_gal, ax=axs[0])
    _, ax = plot2dmap(df_gal, "Number of components", bin_type="default", survey="sami", ax=axs[1], show_title=False, cax_orientation="horizontal")
    _, ax = plot2dmap(df_gal, "BPT (numeric) (total)", bin_type="default", survey="sami", ax=axs[2], show_title=False, cax_orientation="horizontal")
    lat, lon = ax.coords
    lon.set_ticklabel_visible(False)
    ax_sdss.text(s=f"GAMA{gal}", color="white", x=0.1, y=0.9, transform=ax_sdss.transAxes)
    
    if savefigs:
        fname = os.path.join(fig_path, f"matched_sample_2comp_{gal}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_2comp_491956.pdf


In [64]:
# Look at the 1-component galaxies 
N = 4
for gal in gals_1comp[:N]:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))
    bbox = axs[-2].get_position()
    axs[-1].set_position([bbox.x0 + bbox.width, bbox.y0, bbox.width, bbox.height])
    df_gal = df_hq[df_hq.catid == gal]
    ax_sdss = plot_sdss_image(df_gal, ax=axs[0])
    _, ax = plot2dmap(df_gal, "Number of components", bin_type="default", survey="sami", ax=axs[1], show_title=False, cax_orientation="horizontal")
    _, ax = plot2dmap(df_gal, "BPT (numeric) (total)", bin_type="default", survey="sami", ax=axs[2], show_title=False, cax_orientation="horizontal")
    lat, lon = ax.coords
    lon.set_ticklabel_visible(False)
    ax_sdss.text(s=f"GAMA{gal}", color="white", x=0.1, y=0.9, transform=ax_sdss.transAxes)
    
    if savefigs:
        fname = os.path.join(fig_path, f"matched_sample_1comp_{gal}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_1comp_32295.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_1comp_594964.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_1comp_239249.pdf


### STEP 2: select a matched sample 
---
For each galaxy, count how many other galaxies there are in SAMI with 
* $M_*$ within 0.2 dex,
* Total $\rm SFR$ within 0.2 dex, 
* $M_*$ AND total $\rm SFR$ within 0.3 dex.

Other things we might also want to match on:
* number of spaxels with high enough S/N to measure BPT categories 
* angular size, i.e. redshift 
* inclination
* morphology 
* fraction of galaxy light in the SAMI aperture: try SFR surface density rather than total SFR? Compute total SFR within 1R_e

In [10]:
plt.close("all")

In [66]:
# Plot other variables to see what could be driving the number of components present
cols = ["mstar", "SFR (R_e)", "log sigma_SFR (R_e)", "log sSFR (R_e)",
        "R_e (kpc)", "log(M/R_e)", "Morphology (numeric)", "g_i", "m_r", "mu_within_1re",
        "A_V (R_e)", "v_gas (R_e)", "sigma_gas (R_e)",
        "log N2 (R_e)", "log S2 (R_e)", "log O1 (R_e)", "log O3 (R_e)", "S2 ratio (R_e)", 
        "N2S2 (R_e)", "N2O2 (R_e)", "Dopita+2016 (R_e)",
        "Inclination i (degrees)", "z_spec", "kpc per arcsec", "ellip", "pa",
        "Median SNR (B, 1R_e)", "Median SNR (B, 1.5R_e)", "Median SNR (B, 2R_e)", "Median SNR (B, full field)",
        "Median SNR (R, 1R_e)", "Median SNR (R, 1.5R_e)", "Median SNR (R, 2R_e)", "Median SNR (R, full field)"]

In [32]:
# Reduced set of columns for presentation... 
cols = ["mstar", "SFR (R_e)", "log sigma_SFR (R_e)", "log sSFR (R_e)",
        "R_e (kpc)", "log(M/R_e)", "mu_within_1re", "A_V (R_e)", 
        "Inclination i (degrees)", "z_spec", "Median SNR (B, 1R_e)", "Median SNR (R, 1R_e)"]

In [33]:
len(cols)

12

In [40]:
#///////////////////////////////////////////////////////////////////////////
# PARAMETERS
N = 1  # How many galaxies to match to each

#///////////////////////////////////////////////////////////////////////////
# OBTAIN MATCHED SAMPLE
for match_condition in ["mstar", "SFR", "mstar+SFR", "SFR surface density", "mstar+SFR surface density", "sSFR", "mstar+sSFR", "Inclination i (degrees)", "R_e (kpc)", "snr", "kpc per arcsec", "mstar+SFR surface density+kpc per arcsec", "Morphology", "mu_within_1re"]:
# for match_condition in ["kpc per arcsec"]:

    gals_1comp_unmatched = gals_1comp.copy()  # List of 1-component galaxies that haven't been matched yet
    gals_2comp_unmatched = []  # 2-component galaxies that have < N counterparts (and are excluded from analysis)
    gals_1comp_matched = []    # Final list of 1-component galaxies that are counterparts to the 2-component galaxies 
    gals_2comp_matched = []    # Final list of 2-component galaxies that have N counterparts in our sample

    # Split the metadata DataFrame into that containing the 2-component galaxies, and all the others
    df_metadata_2comp = df_metadata_extended.loc[gals_2comp]
    df_metadata_unmatched = df_metadata_extended.loc[gals_1comp_unmatched]
    gals_matched_dict = {}

    # For each 2-comp galaxy,
    for gal in gals_2comp:
        
        # Match conditions
        # Match in mstar
        if match_condition == "mstar":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.2

        # Match in SFR 
        elif match_condition == "SFR":
            sfr = df_metadata_2comp.loc[gal, "SFR (R_e)"]
            cond_matched = np.abs(np.log10(df_metadata_unmatched["SFR (R_e)"]) - np.log10(sfr)) < 0.2

        # Match in mstar & SFR
        elif match_condition == "mstar+SFR":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfr = df_metadata_2comp.loc[gal, "SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3
            cond_matched &= np.abs(np.log10(df_metadata_unmatched["SFR (R_e)"]) - np.log10(sfr)) < 0.3

        # Match in SFR surface density 
        elif match_condition == "SFR surface density":
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.2

        # Match in mstar & SFR surface density
        elif match_condition == "mstar+SFR surface density":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3        
            cond_matched &= np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.3

        # Match in sSFR 
        elif match_condition == "sSFR":
            ssfr = df_metadata_2comp.loc[gal, "log sSFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["log sSFR (R_e)"] - ssfr) < 0.2

        # Match in mstar & sSFR
        elif match_condition == "mstar+sSFR":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            ssfr = df_metadata_2comp.loc[gal, "log sSFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3         
            cond_matched &= np.abs(df_metadata_unmatched["log sSFR (R_e)"] - ssfr) < 0.3

        # Match in inclination
        elif match_condition == "Inclination i (degrees)":
            i = df_metadata_2comp.loc[gal, "Inclination i (degrees)"]
            cond_matched = np.abs(df_metadata_unmatched["Inclination i (degrees)"] - i) < 10

        # Match in R_e 
        elif match_condition == "R_e (kpc)":
            re = df_metadata_2comp.loc[gal, "R_e (kpc)"]
            cond_matched = np.abs(df_metadata_unmatched["R_e (kpc)"] - re) < 1.0
        
        # Match in mstar & Re
        elif match_condition == "mstar+R_e (kpc)":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            re = df_metadata_2comp.loc[gal, "R_e (kpc)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3         
            cond_matched &= np.abs(df_metadata_unmatched["R_e (kpc)"] - re) < 1.0

        # Match in S/N
        elif match_condition == "snr":
            snr = df_metadata_2comp.loc[gal, "Median SNR (B, 1R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["Median SNR (B, 1R_e)"] - snr) < 2

        # Match in kpc per arcsec
        elif match_condition == "kpc per arcsec":
            kpc_per_as = df_metadata_2comp.loc[gal, "kpc per arcsec"]
            cond_matched = np.abs(df_metadata_unmatched["kpc per arcsec"] - kpc_per_as) < 0.3

        # Match in mstar & SFR
        elif match_condition == "mstar+SFR surface density+kpc per arcsec":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            kpc_per_as = df_metadata_2comp.loc[gal, "kpc per arcsec"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3
            cond_matched &= np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.3
            cond_matched &= np.abs(df_metadata_unmatched["kpc per arcsec"] - kpc_per_as) < 0.3

        # Match in surface brightness
        elif match_condition == "mu_1re":
            mu_1re = df_metadata_2comp.loc[gal, "mu_1re"]
            cond_matched = np.abs(df_metadata_unmatched["mu_1re"] - mu_1re) < 0.2
        
        # Match in surface brightness
        elif match_condition == "mu_within_1re":
            mu_1re = df_metadata_2comp.loc[gal, "mu_1re"]
            cond_matched = np.abs(df_metadata_unmatched["mu_1re"] - mu_1re) < 0.2

        # Match in surface brightness
        elif match_condition == "Morphology":
            morph = df_metadata_2comp.loc[gal, "Morphology (numeric)"]
            cond_matched = np.abs(df_metadata_unmatched["Morphology (numeric)"] - morph) == 0
            
        # Match in N2O2
        elif match_condition == "N2O2":
            N2O2 = df_metadata_2comp.loc[gal, "N2O2 (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["N2O2 (R_e)"] - N2O2) < 0.2
            
        else:
            raise ValueError(f"{match_condition} not found!")

        # Get ALL of the galaxies satifying these criteria.
        gals_matched = list(df_metadata_unmatched[cond_matched].index)
        if len(gals_matched) >= N:
            # Select a random subset of these, so that individual galaxies don't "hog" them all
            gals_matched_subset = list(np.random.choice(gals_matched, size=min(N, len(gals_matched)), replace=False))

            # Add the 1-comp galaxies to the list of galaxies that have been matched 
            gals_1comp_matched += gals_matched_subset
            gals_matched_dict[str(gal)] = gals_matched_subset

            # Add the 2-comp galaxy to the list of galaxies that have been matched 
            gals_2comp_matched.append(gal)

            # Remove the newley matched galaxies from the unmatched 1-comp galaxy list 
            df_metadata_unmatched = df_metadata_unmatched.drop(gals_matched_subset, axis=0)
            for gal in gals_matched_subset:
                gals_1comp_unmatched.remove(gal)
        else:
            # Drop this galaxy, because it doesn't have enough counterparts 
            gals_2comp_unmatched.append(gal)
            print(f"{gal} only has {len(gals_matched)} 1-component counterparts")

    # Check that this list is unique so that we aren't accidentally including galaxies twice
    assert len(np.unique(gals_1comp_matched)) == len(gals_1comp_matched)

    #///////////////////////////////////////////////////////////////////////////
    # PLOT 
    # fig, axs = plt.subplots(nrows=5, ncols=7, figsize=(7 * 4, 5 * 4))    
    fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(15, 11))
    for cc, col in enumerate(cols):
        # Plot 
        r = (vmin_fn(col), vmax_fn(col)) if vmin_fn(col) is not None else (df_metadata_extended.loc[gals_2comp_matched + gals_1comp_matched, col].min(), df_metadata_extended.loc[gals_2comp_matched + gals_1comp_matched, col].max())
        hist(df_metadata_extended.loc[gals_2comp_matched, col], histtype="step", ax=axs.flat[cc], range=r, bins=20, label=f"2-component sample (N = {len(gals_2comp_matched)})", normed=False, color="red")
        hist(df_metadata_extended.loc[gals_1comp_matched, col], histtype="stepfilled", alpha=0.3, ax=axs.flat[cc], range=r, bins=20, label=f"Matched sample (N = {len(gals_1comp_matched)})", normed=False, color="orange")

        # Compute mean & std. dev.
        y_2comp = 0.1 * axs.flat[cc].get_ylim()[1]
        y_matched = 0.15 * axs.flat[cc].get_ylim()[1]
        y_whole = 0.2 * axs.flat[cc].get_ylim()[1]
        mean_2comp = df_metadata_extended.loc[gals_2comp_matched, col].mean()
        err_upper_2comp = np.quantile(df_metadata_extended.loc[gals_2comp_matched, col].dropna().values, 0.84) - mean_2comp
        err_lower_2comp = mean_2comp - np.quantile(df_metadata_extended.loc[gals_2comp_matched, col].dropna().values, 0.16)
        axs.flat[cc].errorbar(x=mean_2comp, xerr=[[err_lower_2comp], [err_upper_2comp]], y=y_2comp, marker="o", label="Mean (2-component sample)", color="red")
        
        mean_1comp = df_metadata_extended.loc[gals_1comp_matched, col].mean()
        err_upper_1comp = np.quantile(df_metadata_extended.loc[gals_1comp_matched, col].dropna().values, 0.84) - mean_1comp
        err_lower_1comp = mean_1comp - np.quantile(df_metadata_extended.loc[gals_1comp_matched, col].dropna().values, 0.16)
        axs.flat[cc].errorbar(x=mean_1comp, xerr=[[err_lower_1comp], [err_upper_1comp]], y=y_matched, marker="o", label="Mean (1-component sample)", color="orange")

        axs.flat[cc].set_xlabel(label_fn(col))
        axs.flat[cc].set_axisbelow(True)
        axs.flat[cc].grid(zorder=-1)
        axs.flat[cc].text(x=0.05, y=0.95, transform=axs.flat[cc].transAxes, verticalalignment="top", 
                          s=r"1-comp.: $%.2f ^ {+%.2f} _ {-%.2f}$" % (mean_1comp, err_upper_1comp, err_lower_1comp), color="k")
        axs.flat[cc].text(x=0.05, y=0.87, transform=axs.flat[cc].transAxes, verticalalignment="top", 
                          s=r"2-comp.: $%.2f ^ {%.2f} _ {%.2f}$" % (mean_2comp, err_upper_2comp, err_lower_2comp), color="k")
    axs[0][-1].legend(loc="center left", bbox_to_anchor=[1.1, 0.5])
    title = "Match condition: "
    for s in match_condition.split("+"):
        title += label_fn(s) + ", "
    fig.suptitle(f"{title[:-2]}", y=0.92, fontsize=20)

    # Save 
    if savefigs:
        fname = os.path.join(fig_path, f"matched_sample_hists_{match_condition.replace(' ', '_')}.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

584013 only has 0 1-component counterparts
460374 only has 0 1-component counterparts
572402 only has 0 1-component counterparts
84677 only has 0 1-component counterparts
383259 only has 0 1-component counterparts
3895257 only has 0 1-component counterparts
9011900166 only has 0 1-component counterparts
9016800167 only has 0 1-component counterparts
9016800318 only has 0 1-component counterparts
9239900063 only has 0 1-component counterparts
9239900237 only has 0 1-component counterparts
9239900416 only has 0 1-component counterparts
9388000219 only has 0 1-component counterparts
9388000716 only has 0 1-component counterparts
9008500646 only has 0 1-component counterparts
9403800283 only has 0 1-component counterparts
9403800881 only has 0 1-component counterparts
418737 only has 0 1-component counterparts
9239901557 only has 0 1-component counterparts


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:152: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_hists_mstar.pdf
511867 only has 0 1-component counterparts
64087 only has 0 1-component counterparts
106376 only has 0 1-component counterparts
619095 only has 0 1-component counterparts
543860 only has 0 1-component counterparts
504713 only has 0 1-component counterparts
569555 only has 0 1-component counterparts
485834 only has 0 1-component counterparts
41144 only has 0 1-component counterparts
301382 only has 0 1-component counterparts
136688 only has 0 1-component counterparts
22633 only has 0 1-component counterparts
463288 only has 0 1-component counterparts
298666 only has 0 1-component counterparts
600030 only has 0 1-component counterparts
287824 only has 0 1-component counterparts
491457 only has 0 1-component counterparts
77754 only has 0 1-component counterparts
574029 only has 0 1-component counterparts
584013 only has 0 1-component counterparts
78425 only has 0 1-component counterparts
485833 only has

...

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_hists_Morphology.pdf
491956 only has 0 1-component counterparts
417678 only has 0 1-component counterparts
48740 only has 0 1-component counterparts
85416 only has 0 1-component counterparts
106717 only has 0 1-component counterparts
594906 only has 0 1-component counterparts
514212 only has 0 1-component counterparts
16026 only has 0 1-component counterparts
623272 only has 0 1-component counterparts
570197 only has 0 1-component counterparts
569555 only has 0 1-component counterparts
69462 only has 0 1-component counterparts
136688 only has 0 1-component counterparts
144320 only has 0 1-component counterparts
287824 only has 0 1-component counterparts
491457 only has 0 1-component counterparts
534753 only has 0 1-component counterparts
584013 only has 0 1-component counterparts
47493 only has 0 1-component counterparts
65410 only has 0 1-component counterparts
610474 only has 0 1-component counterparts
93062 only 

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:152: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_hists_mu_within_1re.pdf


## STEP 4: look at the properties of individual 2-component galaxies
---
The goal here is to visually inspect each of our 2-component galaxies in order to check whether they exhibit this "lagging" component. 
Using the LZIFU fits (to check the fluxes in non-Balmer emission lines), make "summary" plots for each galaxy, including maps of emission line fluxes in each component, kinematics and the "lag" diagram.

In [49]:
# For the presentation: SDSS image | v_gas (component 1) | v_gas (component 2) to demonstrate winds/outflows vs. rotation-dominated systems
for gal in [106717, 9011900166]:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(13, 4))
    bbox = axs[-2].get_position()
    axs[-1].set_position([bbox.x0 + bbox.width, bbox.y0, bbox.width, bbox.height])
    df_gal = df_hq[df_hq.catid == gal]
    ax_sdss = plot_sdss_image(df_gal, ax=axs[0])
    _, ax = plot2dmap(df_gal, "v_gas (component 1)", bin_type="default", survey="sami", ax=axs[1], show_title=False, plot_colorbar=False)
    ax.text(s=r"$v_{\rm gas}$ (component 1)", fontsize=15, x=0.05, y=0.95, verticalalignment="top", transform=ax.transAxes)
    _, ax = plot2dmap(df_gal, "v_gas (component 2)", bin_type="default", survey="sami", ax=axs[2], show_title=False, plot_colorbar=True)
    ax.text(s=r"$v_{\rm gas}$ (component 2)", fontsize=15, x=0.05, y=0.95, verticalalignment="top", transform=ax.transAxes)
    lat, lon = ax.coords
    lon.set_ticklabel_visible(False)
    ax_sdss.text(s=f"GAMA{gal}", color="white", x=0.1, y=0.9, transform=ax_sdss.transAxes)
    
    if savefigs:
        fname = os.path.join(fig_path, f"matched_sample_2comp_{gal}_vel.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_2comp_106717_vel.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/matched_sample_2comp_9011900166_vel.pdf


In [ ]:
from spaxelsleuth.loaddata.lzifu import load_lzifu_df
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter

fig_path = "/priv/meggs3/u5708159/SAMI/sami_dr3/matched_samples/summary_plots"

In [ ]:
[g for g in gals_2comp if f"{g}_lag_summary.pdf" not in os.listdir(fig_path)]

In [ ]:
###########################################################################
# Create a collage figure displaying various quantities of interest for 
# this galaxy
###########################################################################
for gal in [g for g in gals_2comp if f"{g}_lag_summary.pdf" not in os.listdir(fig_path)]:
    ###########################################################################
    # Load the LZIFU DataFrame
    # NOTE: NOT using the DF with extinction correction, because it may be a bit dodgy for individual line components. 
    # For the purposes of just checking the fluxes in the emission lines, it doens't really matter anyway.
    ###########################################################################
    try:
        df_gal = load_lzifu_df(gal=gal, 
                               bin_type=bin_type, 
                               ncomponents=ncomponents,
                               eline_SNR_min=eline_SNR_min,
                               correct_extinction=False)  
    except:
        continue

    # layout:
    # SDSS image | number of components | BT classifications
    # delta sigma: 1 | 2 | 3
    # EW: 1 | 2 | 3
    # BPT
    # Emission lines 
    # WHAN | WHAV

    fig, axs = plt.subplots(nrows=10, ncols=3, figsize=(13, 40))
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    axs[-1][-1].set_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # SDSS image 
    ax = plot_sdss_image(df_gal, ax=axs[0][0])
    if ax is not None:
        ax.set_title(f"GAMA{gal}")
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # Number of components
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z="Number of components",
              ax=axs[0][1], 
              plot_colorbar=True, cax=None, cax_orientation="horizontal", 
              show_title=False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    lat = ax.coords[1]
    lat.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # BPT classifications 
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z="BPT (numeric) (total)",
              ax=axs[0][2], 
              plot_colorbar=True, cax=None, cax_orientation="vertical", 
              show_title=False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    lat = ax.coords[1]
    lat.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # v_gas
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"v_gas (component {nn + 1})",
                  ax=axs[1][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=-200, vmax=+200,
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[1][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # sigma_gas
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"sigma_gas (component {nn + 1})",
                  ax=axs[2][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[2][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # HALPHA
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"HALPHA (component {nn + 1})",
                  ax=axs[3][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=0, vmax=2.0,
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[3][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)
    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # [NII]
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"NII6583 (component {nn + 1})",
                  ax=axs[4][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=0, vmax=0.75, cmap="viridis",
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[4][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # [SII]
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"SII6716 (component {nn + 1})",
                  ax=axs[5][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=0, vmax=0.75, cmap="viridis",
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[5][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # HBETA
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"HBETA (component {nn + 1})",
                  ax=axs[6][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=0, vmax=0.75, cmap="viridis",
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[6][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # [OIII]
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"OIII5007 (component {nn + 1})",
                  ax=axs[7][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  vmin=0, vmax=0.75, cmap="viridis",
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[7][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)
        lon = ax.coords[0]
        lon.set_ticklabel_visible(False)

    #///////////////////////////////////////////////////////////////////////////////////////////////////////
    # EW 
    for nn in range(3):
        _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
                  PA_deg=0,
                  col_z=f"HALPHA EW (component {nn + 1})",
                  ax=axs[8][nn], 
                  plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
                  show_title=False)
        ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[8][nn].transAxes, verticalalignment="top")
        if nn > 0:
            lat = ax.coords[1]
            lat.set_ticklabel_visible(False)

    ###########################################################################
    # "Lag" plot
    ###########################################################################
    bbox = axs[-1][0].get_position()
    axs[-1][0].set_position([bbox.x0, bbox.y0 - 0.025, bbox.width, bbox.height])
    bbox = axs[-1][1].get_position()
    axs[-1][1].set_position([bbox.x0 + 0.025, bbox.y0 - 0.025, bbox.width, bbox.height])

    fig = plot2dhistcontours(df_hq,
                             col_x="v_gas (component 1)",
                             col_y="v_gas (component 2)",
                             col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                             xmin=-300, xmax=300, 
                             ymin=-300, ymax=300, 
                             ax=axs[-1][0],
                             plot_colorbar=False,
                             nbins=100)
    fig = plot2dhistcontours(df_hq,
                             col_x="v_*",
                             col_y="v_gas (component 2)",
                             col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                             xmin=-300, xmax=300, 
                             ymin=-300, ymax=300,
                             ax=axs[-1][1],
                             plot_colorbar=False,
                             nbins=100)

    _ = plot2dscatter(df_gal, col_x="v_gas (component 1)", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                      xmin=-300, xmax=300, ymin=-300, ymax=300, 
                      vmax=150,
                      plot_colorbar=False,
                      ax=axs[-1][0])

    _ = plot2dscatter(df_gal, col_x="v_*", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                      xmin=-300, xmax=300, ymin=-300, ymax=300, 
                      vmax=150,
                      plot_colorbar=True,
                      ax=axs[-1][1])
    cax = fig.get_axes()[-1]

    # Decorations 
    [ax.grid() for ax in axs[-1]]
    [ax.set_xlabel(ax.get_xlabel() + " (component 1)") for ax in axs[-1]]
    axs[-1][0].set_ylabel(axs[-1][0].get_ylabel() + " (component 2)")
    axs[-1][1].set_ylabel("")
    cax.set_ylabel(cax.get_ylabel() + " (component 2)")
    # axs[-1][1].set_yticklabels([])
    [ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--") for ax in axs[-1]]
    [ax.plot([-300, 300], [0, 0], color="k") for ax in axs[-1]]
    [ax.plot([0, 0], [-300, 300], color="k") for ax in axs[-1]]
    axs[-1][0].axvline(df_gal.loc[(df_gal["x (projected, arcsec)"] == 12.5) & (df_gal["y (projected, arcsec)"] == 12.5), "v_gas (component 1)"].values[0], ls="--", label=r"$v_{\rm gas\, (centre)}$")
    axs[-1][0].axhline(df_gal.loc[(df_gal["x (projected, arcsec)"] == 12.5) & (df_gal["y (projected, arcsec)"] == 12.5), "v_gas (component 2)"].values[0], ls="--")
    axs[-1][0].legend()
    axs[-1][1].axhline(df_gal.loc[(df_gal["x (projected, arcsec)"] == 12.5) & (df_gal["y (projected, arcsec)"] == 12.5), "v_gas (component 2)"].values[0], ls="--", label=r"$v_{\rm gas\, (centre)}$")
    axs[-1][1].axvline(df_gal.loc[(df_gal["x (projected, arcsec)"] == 12.5) & (df_gal["y (projected, arcsec)"] == 12.5), "v_*"].values[0], ls=":", label=r"$v_{\rm *\, (centre)}$")
    axs[-1][1].legend()

    # Save 
    fig.savefig(os.path.join(fig_path, f"{gal}_lag_summary.pdf"), format="pdf", bbox_inches="tight")
    plt.close("all")
        

## Step 5: "lag" plots 
---
First: make the lag plot for the *whole* SAMI sample and then just for the 2comp sample. Do they look similar?

In [9]:
plt.rcParams.update({'font.size': 14})

In [10]:
cond = np.zeros(df_hq.shape[0], dtype="bool")
for gal in gals_2comp:
    cond |= (df_hq["BPT (total)"] == "SF") & (df_hq["catid"] == gal)
df_SF_2comp = df_hq[cond]
df_SF = df_hq[df_hq["BPT (total)"] == "SF"]

In [12]:
fig = plot2dhistcontours(df_SF_2comp,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=True, 
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [-300 + 50, 300 + 50], color="k", lw=1, ls="--")
ax.plot([-300, 300], [-300 - 50, 300 - 50], color="k", lw=1, ls="--")

# Save 
if savefigs:
    fname = os.path.join(fig_path, f"lag_2comp_vgas_2_vs_1.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/lag_2comp_vgas_2_vs_1_blank.pdf


In [75]:
fig = plot2dhistcontours(df_SF_2comp,
                         col_x="v_*",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=True, 
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [-300 + 50, 300 + 50], color="k", lw=1, ls="--")
ax.plot([-300, 300], [-300 - 50, 300 - 50], color="k", lw=1, ls="--")

# Save 
if savefigs:
    fname = os.path.join(fig_path, f"lag_2comp_vgas_2_vs_vstar.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/lag_2comp_vgas_2_vs_vstar.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


In [77]:
fig = plot2dhistcontours(df_SF_2comp,
                         col_y="v_*",
                         col_x="v_gas (component 1)",
                         col_z="count", log_z=True, 
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [-300 + 50, 300 + 50], color="k", lw=1, ls="--")
ax.plot([-300, 300], [-300 - 50, 300 - 50], color="k", lw=1, ls="--")

# Save 
if savefigs:
    fname = os.path.join(fig_path, f"lag_2comp_vgas_1_vs_vstar.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/lag_2comp_vgas_1_vs_vstar.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


In [13]:
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, vmin_fn, vmax_fn, cmap_fn, label_fn, fname_fn, bpt_labels, bpt_colours, ncomponents_colours, ncomponents_labels, component_colours, component_labels
plt.rcParams.update({'font.size': 12})
for col in ["sigma_gas", "v_gas", "log HALPHA EW"]:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
    for nn in range(1, 4):
        cond = ~df_SF_2comp[f"{col} (component {nn})"].isna()
        n_comp = df_SF_2comp[cond].shape[0]
        hist(df_SF_2comp[f"{col} (component {nn})"], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=component_colours[nn - 1], ax=ax, normed=True, label=f"Component {nn} ($N$ = {n_comp})")
    ax.set_xlabel(label_fn(col))
    ax.set_ylabel("N (normalised)")
    ax.legend(fontsize=10, loc="lower center", bbox_to_anchor=[0.5, 1.05])
    ax.grid()
    
    if savefigs:
        fname = os.path.join(fig_path, f"hist_{fname_fn(col)}_2comp.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_sigma_gas_2comp.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_v_gas_2comp.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_logHaEW_2comp.pdf


In [16]:
# Re-make the above plots but overlaid with points from individual galaxies 
from spaxelsleuth.plotting.plotgalaxies import plot2dscatter
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         plot_colorbar=False,
                         nbins=50)
ax = fig.get_axes()[0]
 
gal = 460374
df_gal = df_hq[df_hq["catid"] == gal]
_ = plot2dscatter(df_gal, col_x="v_gas (component 1)", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                  xmin=-300, xmax=300, ymin=-300, ymax=300, vmax=150,
                  plot_colorbar=True,
                  ax=ax)

# Decorations 
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [0, 0], color="k")
ax.plot([0, 0], [-300, 300], color="k")

# SDSS image
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
plot_sdss_image(df_gal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …